In [1]:
import pandas as pd

from toolz import *

import json
import pathlib

import glom
from lenses import lens

In [2]:
lmap = compose(list, map)

In [3]:
def list_of_str_to_list_of_json(lst):
    it = map(lambda x: x.replace('\xa0', ' '), lst)
    it = filter(lambda x: x != '', it)
    it = lmap(json.loads, it)
    return it

In [4]:
data_path = pathlib.Path('data')

In [5]:
instance_entry_lens_get = lens['Завтраки'][3].get()

## Loading dataset from files (as nested structure):

In [6]:
dict_json_by_course = {}
for path_course in data_path.joinpath('collected').iterdir():
    list_of_course_jsons = []
    for path_file in path_course.iterdir():
        with open(path_file, 'r', encoding='utf-8') as f:
            list_of_course_jsons += list_of_str_to_list_of_json(f.readlines())
    dict_json_by_course[str(path_course.name)] = list_of_course_jsons

In [7]:
dict_json_by_course.keys()

dict_keys(['Выпечка и десерты', 'Завтраки', 'Закуски', 'Напитки', 'Основные блюда', 'Паста и пицца', 'Салаты', 'Соусы и маринады', 'Супы'])

In [9]:
instance_entry_lens_get(dict_json_by_course)

{'url': 'https://eda.ru/recepty/zavtraki/zapekanka-iz-lapshi-vetchini-schinkenfleckerln-22024',
 'title': 'Запеканка из лапши и ветчины (Schinkenfleckerln)',
 'category': 'Завтраки',
 'cuisine': 'Австрийская кухня',
 'menu': 'Еда без ограничений',
 'description': 'Запеканка из макарон — вовсе не странность: ашкеназский кугель, русский лапшевик, итальянская лазанья, листы для которой тоже относятся к пасте, так что австрийский шинкенфлекерльн в хорошей компании. Приготовить это блюдо проще, чем выговорить его название: сочетание лапши, ветчины и сыра, конечно, сплошные калории, зато ученые все больше склоняются к тому, что жиры и углеводы действительно могут помочь нам справиться со стрессом. Или можете заменить жирные сорта ветчины, к примеру, вареной индейкой, сразу сбросив часть груза калорий. К тому же готовится такая запеканка не дольше, чем яичница, вполне укладывается даже в утренний тайминг и позволяет пустить в дело замыленный кусочек сыра или остатки вчерашних макарон. Если бу

## Processing:

Now we're ready to process this data to create dataset for our needs

In [11]:
# lens that yields seperate dicts from the data
# (like `instance_entry_lens_get(dict_json_by_course)` above)
data_processing_lens = lens.Values().Each()

### First of all, renaming `category` to `course`:

In [13]:
dict_json_by_course_renamed = data_processing_lens.Keys().modify(
    lambda field_name: 'course' if field_name == 'category' else field_name,
)(dict_json_by_course)

example_instance = instance_entry_lens_get(dict_json_by_course_renamed)
example_instance

{'url': 'https://eda.ru/recepty/zavtraki/zapekanka-iz-lapshi-vetchini-schinkenfleckerln-22024',
 'title': 'Запеканка из лапши и ветчины (Schinkenfleckerln)',
 'course': 'Завтраки',
 'cuisine': 'Австрийская кухня',
 'menu': 'Еда без ограничений',
 'description': 'Запеканка из макарон — вовсе не странность: ашкеназский кугель, русский лапшевик, итальянская лазанья, листы для которой тоже относятся к пасте, так что австрийский шинкенфлекерльн в хорошей компании. Приготовить это блюдо проще, чем выговорить его название: сочетание лапши, ветчины и сыра, конечно, сплошные калории, зато ученые все больше склоняются к тому, что жиры и углеводы действительно могут помочь нам справиться со стрессом. Или можете заменить жирные сорта ветчины, к примеру, вареной индейкой, сразу сбросив часть груза калорий. К тому же готовится такая запеканка не дольше, чем яичница, вполне укладывается даже в утренний тайминг и позволяет пустить в дело замыленный кусочек сыра или остатки вчерашних макарон. Если буде

### Get rid of redundant data and generate `id`s:

In [11]:
id_from_url = lambda url: int(url.split('-')[-1])
def process_instance(instance):
    return glom.glom(instance, {
        'id': glom.Invoke(id_from_url).specs('url'),
        'title': glom.Invoke(str.strip).specs('title'),
        'course': 'course',
        'cuisine': 'cuisine',
        'ingredients': ('ingredients', glom.Iter().map(lambda x: x[0].strip()).all())
    })
process_instance(example_instance)

{'id': 22024,
 'title': 'Запеканка из лапши и ветчины (Schinkenfleckerln)',
 'course': 'Завтраки',
 'cuisine': 'Австрийская кухня',
 'ingredients': ['Ветчина',
  'Широкая лапша',
  'Репчатый лук',
  'Тертый сыр пармезан',
  'Соль',
  'Свежемолотый черный перец',
  'Оливковое масло']}

In [12]:
dict_json_by_course_processed = data_processing_lens.modify(process_instance)(dict_json_by_course_renamed)
instance_entry_lens_get(dict_json_by_course_processed)

{'id': 22024,
 'title': 'Запеканка из лапши и ветчины (Schinkenfleckerln)',
 'course': 'Завтраки',
 'cuisine': 'Австрийская кухня',
 'ingredients': ['Ветчина',
  'Широкая лапша',
  'Репчатый лук',
  'Тертый сыр пармезан',
  'Соль',
  'Свежемолотый черный перец',
  'Оливковое масло']}

### We're ready to flatten our data and turn it into dataframe:

In [13]:
all_instances_flattened = data_processing_lens.collect()(dict_json_by_course_processed)
all_instances_flattened[0]

{'id': 28195,
 'title': 'Банановое мороженое с корицей',
 'course': 'Выпечка и десерты',
 'cuisine': 'Карибская кухня',
 'ingredients': ['Бананы',
  'Лимонный сок',
  'Молотая корица',
  'Ванильное мороженое']}

In [14]:
df = pd.DataFrame(all_instances_flattened)
df['ingredients'] = df['ingredients'].map(tuple)
df

,id,title,course,cuisine,ingredients
0,28195,Банановое мороженое с корицей,Выпечка и десерты,Карибская кухня,"(Бананы, Лимонный сок, Молотая корица, Ванильн..."
1,28141,Ананасово-кокосовый шербет,Выпечка и десерты,Карибская кухня,"(Сахар, Кокосовое молоко, Корень имбиря, Анана..."
2,27929,"Десерт из манго, клубники и текилы",Выпечка и десерты,Карибская кухня,"(Клубника, Манго, Сахар, Тертая цедра лайма, Т..."
3,28192,Ананас с соусом из манго и рома,Выпечка и десерты,Карибская кухня,"(Манго, Темный ром, Сок лайма, Сахар, Тертая ц..."
4,28463,Клубничный соус с текилой,Выпечка и десерты,Карибская кухня,"(Клубника, Сахарная пудра, Сок лайма, Текила, ..."
...,...,...,...,...,...
42196,43380,Постный борщ с фасолью,Супы,Украинская кухня,"(Белая фасоль, Овощной бульон, Свекла, Картофе..."
42197,80446,Суп энгамат,Супы,Шведская кухня,"(Цветная капуста, Морковь, Картофель, Лук-поре..."
42198,136820,Гороховый суп с блинчиками,Супы,Шведская кухня,"(Горох, Репчатый лук, Гвоздика, Свиная рулька,..."
42199,18014,Суп из брокколи и кростини с сыром бри,Супы,Шведская кухня,"(Оливковое масло, Репчатый лук, Овощной бульон..."


In [15]:
dups = df.duplicated()
print(f'there were {dups.sum()} duplicate rows!')
df_without_duplicates = df[~dups].reset_index(drop=True)
df_without_duplicates

there were 763 duplicate rows!


,id,title,course,cuisine,ingredients
0,28195,Банановое мороженое с корицей,Выпечка и десерты,Карибская кухня,"(Бананы, Лимонный сок, Молотая корица, Ванильн..."
1,28141,Ананасово-кокосовый шербет,Выпечка и десерты,Карибская кухня,"(Сахар, Кокосовое молоко, Корень имбиря, Анана..."
2,27929,"Десерт из манго, клубники и текилы",Выпечка и десерты,Карибская кухня,"(Клубника, Манго, Сахар, Тертая цедра лайма, Т..."
3,28192,Ананас с соусом из манго и рома,Выпечка и десерты,Карибская кухня,"(Манго, Темный ром, Сок лайма, Сахар, Тертая ц..."
4,28463,Клубничный соус с текилой,Выпечка и десерты,Карибская кухня,"(Клубника, Сахарная пудра, Сок лайма, Текила, ..."
...,...,...,...,...,...
41433,43380,Постный борщ с фасолью,Супы,Украинская кухня,"(Белая фасоль, Овощной бульон, Свекла, Картофе..."
41434,80446,Суп энгамат,Супы,Шведская кухня,"(Цветная капуста, Морковь, Картофель, Лук-поре..."
41435,136820,Гороховый суп с блинчиками,Супы,Шведская кухня,"(Горох, Репчатый лук, Гвоздика, Свиная рулька,..."
41436,18014,Суп из брокколи и кростини с сыром бри,Супы,Шведская кухня,"(Оливковое масло, Репчатый лук, Овощной бульон..."


### As one can see, ids don't repeat:

In [16]:
it = df_without_duplicates.groupby('id').count()
it = it[it['course'] > 1]
len(it.index)

0

In [17]:
df_without_duplicates.to_json(data_path.joinpath('ready_dataframe.json'),force_ascii=False)